# Import necessarry Libraries

In [1]:
import gym
import numpy as np
from gym import wrappers
import random as rand
import seaborn as sns
import matplotlib.pyplot as plt
import time
from IPython.display import clear_output 
#-----set up environment------------------------------------#
env = gym.make('FrozenLake-v0')

# Set up environment variables

In [2]:
MAX_STATES = 15
GAMMA = 0.8
ALPHA =0.01

def get_state_as_string(env_state):
    env_string = ''.join(str(int(e)) for e in env_state)
    return env_string

def get_all_states_string():
    env_states = []
    for i in range(MAX_STATES):
        env_states.append(str(i).zfill(0))
    return env_states

def initialise_q():
    Q = {}
    all_states = get_all_states_string()
    for state in all_states:
        Q[state] = {}
        for action in range(env.action_space.n):
            Q[state][action] = 0 #set rewards
    return Q

def add_state_to_table(new_state,table):
    n_value = {}
    n_value[new_state] = {}
    for action in range(env.action_space.n):
        n_value[new_state][action] = 0
    table.update(n_value)

def q_states_dict(q):
    a,b = max(q.items(), key=lambda k: k[1])
    #print("z",a,b)
    return a,b

def play_a_game(Q,exp_rate):
    observation = env.reset()
    done = False
    count = 0
    new_states_created = 0
    total_reward = 0
    state = str(observation)
    y = Q.get(state)
    if y == None:
        add_state_to_table(state,Q)
        new_states_created+=1
    while not done:
        count+=1
        init_random = np.random.uniform(0,1)
        exp_thresh = rand.uniform(0,1)
        if exp_thresh > exp_rate:
            act,_ = q_states_dict(Q[state])
        else:
            act = env.action_space.sample()
        observation,reward,done,_ = env.step(act)
        total_reward+=reward
        new_state = str(observation)
        y = Q.get(new_state)
        if y == None:
            new_states_created+=1
            add_state_to_table(new_state,Q)
        a1,max_q_s1a1 = q_states_dict(Q[new_state])
        Q[state][act]= ((1 - ALPHA)*(Q[state][act])) + ALPHA*(reward + GAMMA * max_q_s1a1)
        #state,act = new_state,a1
        if done:
            if reward == 0:
                reward = -0.0625
                Q[state][act]= ((1 - ALPHA)*(Q[state][act])) + ALPHA*(reward + GAMMA * max_q_s1a1)
            elif reward == 1:
                reward = 1+0.0625
                Q[state][act]= ((1 - ALPHA)*(Q[state][act])) + ALPHA*(reward + GAMMA * max_q_s1a1)
            q_t = Q
            break
        state,act = new_state,a1
    return total_reward, count,new_states_created,q_t

def play_multiple(N):
    Q = initialise_q()
    count=[]
    new_state_array = 0
    reward = []
    exp_rate = 1
    episode = 0
    max_rate = 1
    min_rate = 0.01
    exp_decay = 0.000001
    for n in range(N):
        ep_reward,ep_length,states_created,q_t = play_a_game(Q,exp_rate)
        episode+=1
        exp_rate = min_rate + (max_rate - min_rate)*np.exp(-exp_decay * episode)
        new_state_array+=states_created
        if n %10000 == 0:
            print(n,ep_reward,new_state_array)
        count.append(ep_length)
        reward.append(ep_reward)
    return count,reward,q_t

# Training

In [5]:
x,y,q_t = play_multiple(100000)#number of games wished to play/train on

0 0.0 0
10000 0.0 1
20000 0.0 1
30000 0.0 1
40000 0.0 1
50000 0.0 1
60000 0.0 1
70000 0.0 1
80000 0.0 1
90000 0.0 1


## View Q table data

In [1]:
q_t

NameError: name 'q_t' is not defined

In [8]:
# Test Learned values against environment

1558.0

In [9]:
wins = 0
loses = 0
numberof games = 3
for episode in range (3):
    state = env.reset()
    done = False
    print("-----------------------")
    time.sleep(1)
    for step in range(150):
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)
        state = str(state)
        act,_ = q_states_dict(q_t[state])
        new_sts,reward,done,_ = env.step(act)
        
        if done:
            clear_output(wait=True)
            env.render()
            if reward == 1:
                print("Found Goal!")
                wins+=1
                time.sleep(3)
            else:
                print("You Lose")
                loses+=1
                time.sleep(3)
            clear_output(wait=True)
            break
            
        state = new_sts

print("wins: ",wins," loses: ",loses)
env.close()  

wins:  3  loses:  0


# Deep Q

In [ ]:
#replay memory is finite
#poicy network
